# Análisis de vías rurales

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Cargar librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from EDA_cleaning_utils import tabla_frecuencia
from EDA_cleaning_utils import graficar_barras
from EDA_cleaning_utils import normalizar_columna
import hvplot.pandas
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv('collected_obra_data_all_process.csv')
df.head()

,CONTRACT_ID,ENTITY_NAME,DEPARTMENT,MUNICIPALITY_TYPE,PROCESS_TYPE,CONTRACT_OBJECT,OBJETC_DETAIL,ESTIMATED_COST_ORIG,CONTRACT_VALUE_ORIG,ADDITIONAL_COST_ORIG,FINAL_COST_ORIG,YEAR,CONTRACT_DATE,START_DATE,ORIGINAL_DEADLINE,END_DATE,ID_ADJUDICACION,URLPROCESO,NIT_ENTIDAD,ID_CONTRATISTA,ESTIMATED_COST_NORM,CONTRACT_VALUE_NORM,ADDITIONAL_COST_NORM,FINAL_COST_NORM,ADDITIONAL_TIME,FINAL_DEADLINE,PROJECT_INTENSITY_NORM,PROJECT_INTENSITY_ORIG,AWARD_GROWTH_NORM,AWARD_GROWTH_ORIG,COST_DEVIATION_ORIG,COST_DEVIATION_NORM,TIME_DEVIATION,OWNER,REGION,GROUP,COST_GROUP_QUANTILE
0,17-13-6626390-6041040,NARINO ALCALDIA MUNICIPIO DE ARBOLEDA BERRUECOS,NARIÑO,TYPE_2,CONTRATACIÓN MÍNIMA CUANTÍA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONTRATAR LA OBRA PUBLICA PARA REALIZAR EL MAN...,1.999626e+07,1.999626e+07,0.0,1.999626e+07,2017,2017-05-26,2017-05-26,7,2017-06-02,6041040,{'url': 'https://www.contratos.gov.co/consulta...,800093386-8,1018464415,27.105601,27.105601,0.000000,27.105601,0,7,3.872229,2.856609e+06,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,PACIFICA,Vias Rurales hasta 2020,1
1,19-1-205607-10520942,ANTIOQUIA ALCALDIA MUNICIPIO DE EL PENOL,ANTIOQUIA,TYPE_6,LICITACIÓN PÚBLICA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",FRESADA Y PAVIMENTACION DE LA VIA DE ACCESO PR...,1.104422e+09,1.084239e+09,250060500.0,1.334300e+09,2019,2019-11-28,2019-12-04,60,2020-05-04,10520942,{'url': 'https://www.contratos.gov.co/consulta...,890980917,901337744,1333.656541,1309.283977,301.963131,1611.247108,90,150,21.821400,1.807065e+07,-1.827499,-1.827499,0.230632,0.230632,1.5,MUNICIPALITY,ANDINA,Vias Rurales hasta 2020,4
2,19-13-8824649-8190165,ANTIOQUIA ALCALDIA MUNICIPIO DE NECOCLI,ANTIOQUIA,TYPE_6,CONTRATACIÓN MÍNIMA CUANTÍA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",MANTENIMIENTO VIA ACCESO AL COLISEO DEL MUNIC...,2.150000e+07,2.150000e+07,0.0,2.150000e+07,2019,2019-01-03,2019-01-22,10,2019-02-01,8190165,{'url': 'https://www.contratos.gov.co/consulta...,890985354-8,900821698,25.962546,25.962546,0.000000,25.962546,0,10,2.596255,2.150000e+06,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,ANDINA,Vias Rurales hasta 2020,1
3,19-12-9611888-8756837,CAUCA ALCALDIA MUNICIPIO DE PIAMONTE,CAUCA,TYPE_6,CONTRATACIÓN DIRECTA (LEY 1150 DE 2007),"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",APOYO AL CONVENIO SOLIDARIO PARA EL MEJORAMIEN...,2.000000e+07,2.000000e+07,0.0,2.000000e+07,2019,2019-06-26,2019-06-26,30,2019-07-26,8756837,{'url': 'https://www.contratos.gov.co/consulta...,891856464-0,901276910,24.151206,24.151206,0.000000,24.151206,0,30,0.805040,6.666667e+05,0.000000,0.000000,0.000000,0.000000,0.0,MUNICIPALITY,PACIFICA,Vias Rurales hasta 2020,1
4,19-13-9683074-9182713,ANTIOQUIA ALCALDIA MUNICIPIO DE NECOCLI,ANTIOQUIA,TYPE_6,CONTRATACIÓN MÍNIMA CUANTÍA,"TERRENOS, EDIFICIOS, ESTRUCTURAS Y VÍAS",CONSTRUCCION BOXCOULVER A TODO COSTO EN EL CEN...,2.300000e+07,2.299620e+07,0.0,2.299620e+07,2019,2019-07-12,2019-09-25,30,2019-10-25,9182713,{'url': 'https://www.contratos.gov.co/consulta...,890985354-8,900821698,27.773887,27.769297,0.000000,27.769297,0,30,0.925643,7.665400e+05,-0.016526,-0.016526,0.000000,0.000000,0.0,MUNICIPALITY,ANDINA,Vias Rurales hasta 2020,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6790 entries, 0 to 6789
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CONTRACT_ID             6790 non-null   object 
 1   ENTITY_NAME             6790 non-null   object 
 2   DEPARTMENT              6790 non-null   object 
 3   MUNICIPALITY_TYPE       6790 non-null   object 
 4   PROCESS_TYPE            6790 non-null   object 
 5   CONTRACT_OBJECT         6790 non-null   object 
 6   OBJETC_DETAIL           6790 non-null   object 
 7   ESTIMATED_COST_ORIG     6790 non-null   float64
 8   CONTRACT_VALUE_ORIG     6790 non-null   float64
 9   ADDITIONAL_COST_ORIG    6790 non-null   float64
 10  FINAL_COST_ORIG         6790 non-null   float64
 11  YEAR                    6790 non-null   int64  
 12  CONTRACT_DATE           6790 non-null   object 
 13  START_DATE              6784 non-null   object 
 14  ORIGINAL_DEADLINE       6790 non-null   

## Procesamiento básico

In [5]:
df['CONTRACT_VALUE_ORIG_MILLIONS'] = df['CONTRACT_VALUE_ORIG']/1e6
df['ADDITIONAL_COST_ORIG_MILLIONS'] = df['ADDITIONAL_COST_ORIG']/1e6
df['FINAL_COST_ORIG_MILLIONS'] = df['FINAL_COST_ORIG']/1e6
df['OBJETC_DETAIL'] = df['OBJETC_DETAIL'].str.upper()

## Filtros

In [6]:
df.groupby('GROUP').size()

GROUP
Vias Rurales 2021-2023                      651
Vias Rurales en SECOP II 2021-2023          127
Vias Rurales en SECOP II hasta 2020          30
Vias Rurales hasta 2020                    4286
Vias secundarias 2021-2023                   97
Vias secundarias en SECOP II 2021-2023       21
Vias secundarias en SECOP II hasta 2020       9
Vias secundarias hasta 2020                1569
dtype: int64

In [7]:
# Quitar vías secundarías
df = df.loc[df['GROUP'].str.contains('Vias Rurales')].copy()

In [8]:
df.shape

(5094, 40)

In [9]:
df.groupby('GROUP').size()

GROUP
Vias Rurales 2021-2023                  651
Vias Rurales en SECOP II 2021-2023      127
Vias Rurales en SECOP II hasta 2020      30
Vias Rurales hasta 2020                4286
dtype: int64

In [10]:
df['OBJETC_DETAIL'] = df['OBJETC_DETAIL'].str.upper()

In [11]:
# Quitar convenios interadministrativos
kw_convenios = [
    'AUNAR',
    'CONVENIO',
    'ANUAR',
    'AUNNAR',
    'INTERADMINISTRATIVO',
    'COMPLEMENTACION DE ESFUERZOS INSTITUCIONALES',
    'COMPLEMENTAR ESFUERZOS INSTITUCIONALES',
    'UNIR  ESFUERZOS',
    'UNIR ESFUERZOS',
    'UNION DE ESFUERZOS'
               ]
# Unir las palabras clave en un solo string separadas por '|'
kw_convenios = '|'.join(kw_convenios)
df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_convenios)].copy()
df.shape

(4537, 40)

In [12]:
# Quitar contratos que no son obra
kw_otros = ['ESTUDIOS',
             'DISENOS',
             'ESTUDIO',
             'DISENO',
             'INTERVENTORIA',
             'INTERVENTORA',
             'INTRERVENTORIA',
             'ADMINISTRACION',
             'INVENTARIO',
             'ELABORACION DE UN  MANUAL',
             'SUMINISTRO',
             'INSTITUCIONES EDUCATIVAS',
             'INSTITUCION EDUCATIVA',
             'MANTENIMIENTO RUTINARIO'
             ]

kw_otros = '|'.join(kw_otros)

df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_otros)].copy()
df.shape

(4102, 40)

In [13]:
# Quitar otros tipos de obras
kw_otras_obras = ['ACUEDUCTO',
                  'ALCANTARILLADO',
                  'PARQUE',
                  'COLEGIO']

kw_otras_obras = '|'.join(kw_otras_obras)

df = df.loc[~df['OBJETC_DETAIL'].str.contains(kw_otras_obras)].copy()
df.shape

(3995, 40)

In [14]:
df.groupby('PROCESS_TYPE').size()

PROCESS_TYPE
CONCURSO DE DISEÑO ARQUITECTÓNICO                                                           1
CONCURSO DE MÉRITOS ABIERTO                                                                 1
CONTRATACIÓN DIRECTA                                                                       94
CONTRATACIÓN DIRECTA (CON OFERTAS)                                                          6
CONTRATACIÓN DIRECTA (LEY 1150 DE 2007)                                                  1430
CONTRATACIÓN MÍNIMA CUANTÍA                                                              1071
CONTRATACIÓN RÉGIMEN ESPECIAL                                                               1
CONTRATOS Y CONVENIOS CON MÁS DE DOS PARTES                                                 2
LICITACIÓN OBRA PÚBLICA                                                                    92
LICITACIÓN PÚBLICA                                                                        359
LICITACIÓN PÚBLICA OBRA PUBLICA                

In [15]:
modalidad_consulta = ['CONCURSO DE DISEÑO ARQUITECTÓNICO','CONCURSO DE MÉRITOS ABIERTO','CONTRATOS Y CONVENIOS CON MÁS DE DOS PARTES']
col_sel = ['OBJETC_DETAIL','CONTRACT_VALUE_ORIG_MILLIONS']
df.loc[df['PROCESS_TYPE'].isin(modalidad_consulta),col_sel].to_clipboard()

In [16]:
df.columns

Index(['CONTRACT_ID', 'ENTITY_NAME', 'DEPARTMENT', 'MUNICIPALITY_TYPE',
       'PROCESS_TYPE', 'CONTRACT_OBJECT', 'OBJETC_DETAIL',
       'ESTIMATED_COST_ORIG', 'CONTRACT_VALUE_ORIG', 'ADDITIONAL_COST_ORIG',
       'FINAL_COST_ORIG', 'YEAR', 'CONTRACT_DATE', 'START_DATE',
       'ORIGINAL_DEADLINE', 'END_DATE', 'ID_ADJUDICACION', 'URLPROCESO',
       'NIT_ENTIDAD', 'ID_CONTRATISTA', 'ESTIMATED_COST_NORM',
       'CONTRACT_VALUE_NORM', 'ADDITIONAL_COST_NORM', 'FINAL_COST_NORM',
       'ADDITIONAL_TIME', 'FINAL_DEADLINE', 'PROJECT_INTENSITY_NORM',
       'PROJECT_INTENSITY_ORIG', 'AWARD_GROWTH_NORM', 'AWARD_GROWTH_ORIG',
       'COST_DEVIATION_ORIG', 'COST_DEVIATION_NORM', 'TIME_DEVIATION', 'OWNER',
       'REGION', 'GROUP', 'COST_GROUP_QUANTILE',
       'CONTRACT_VALUE_ORIG_MILLIONS', 'ADDITIONAL_COST_ORIG_MILLIONS',
       'FINAL_COST_ORIG_MILLIONS'],
      dtype='object')

## Procesamiento

In [17]:
df.groupby('PROCESS_TYPE').size()

PROCESS_TYPE
CONCURSO DE DISEÑO ARQUITECTÓNICO                                                           1
CONCURSO DE MÉRITOS ABIERTO                                                                 1
CONTRATACIÓN DIRECTA                                                                       94
CONTRATACIÓN DIRECTA (CON OFERTAS)                                                          6
CONTRATACIÓN DIRECTA (LEY 1150 DE 2007)                                                  1430
CONTRATACIÓN MÍNIMA CUANTÍA                                                              1071
CONTRATACIÓN RÉGIMEN ESPECIAL                                                               1
CONTRATOS Y CONVENIOS CON MÁS DE DOS PARTES                                                 2
LICITACIÓN OBRA PÚBLICA                                                                    92
LICITACIÓN PÚBLICA                                                                        359
LICITACIÓN PÚBLICA OBRA PUBLICA                

In [18]:
tipo_proceso_mod = ['CONTRATACIÓN DIRECTA','LICITACIÓN','SELECCIÓN ABREVIADA','MÍNIMA CUANTÍA','RÉGIMEN ESPECIAL']
df['PROCESS_TYPE_MOD'] = ''
for i in tipo_proceso_mod:
    df.loc[df['PROCESS_TYPE'].str.contains(i), ['PROCESS_TYPE_MOD']] = i
df.loc[df['PROCESS_TYPE_MOD'] == '', ['PROCESS_TYPE_MOD']] = 'OTRO'

In [19]:
# Estandarizar la modalidad de contratación
#modalidad_map = {
#    'LICITACIÓN PÚBLICA':'LICITACIÓN', 
#    'CONTRATACIÓN DIRECTA (LEY 1150 DE 2007)':'CONTRATACION DIRECTA',
#    'SELECCIÓN ABREVIADA DE MENOR CUANTÍA (LEY 1150 DE 2007)':'SELECCIÓN ABREVIADA',
#    'LICITACIÓN OBRA PÚBLICA':'LICITACIÓN', 
#    'SELECCIÓN ABREVIADA DEL LITERAL H DEL NUMERAL 2 DEL ARTÍCULO 2 DE LA LEY 1150 DE 2007':'SELECCIÓN ABREVIADA'
#}
#df['PROCESS_TYPE'].replace(modalidad_map, inplace=True)

In [20]:
df.groupby('PROCESS_TYPE_MOD').size().sort_values(ascending=False)

PROCESS_TYPE_MOD
CONTRATACIÓN DIRECTA    1530
MÍNIMA CUANTÍA          1088
SELECCIÓN ABREVIADA      617
LICITACIÓN               455
RÉGIMEN ESPECIAL         301
OTRO                       4
dtype: int64

In [21]:
df.isnull().sum()

CONTRACT_ID                       0
ENTITY_NAME                       0
DEPARTMENT                        0
MUNICIPALITY_TYPE                 0
PROCESS_TYPE                      0
CONTRACT_OBJECT                   0
OBJETC_DETAIL                     0
ESTIMATED_COST_ORIG               0
CONTRACT_VALUE_ORIG               0
ADDITIONAL_COST_ORIG              0
FINAL_COST_ORIG                   0
YEAR                              0
CONTRACT_DATE                     0
START_DATE                        4
ORIGINAL_DEADLINE                 0
END_DATE                          4
ID_ADJUDICACION                   0
URLPROCESO                        0
NIT_ENTIDAD                       0
ID_CONTRATISTA                    0
ESTIMATED_COST_NORM               0
CONTRACT_VALUE_NORM               0
ADDITIONAL_COST_NORM              0
FINAL_COST_NORM                   0
ADDITIONAL_TIME                   0
FINAL_DEADLINE                    0
PROJECT_INTENSITY_NORM            0
PROJECT_INTENSITY_ORIG      

In [22]:
dep_mod = {'BOGOT DC':'BOGOTÁ DC',
            'BOYAC':'BOYACÁ',
            'CAQUET':'CAQUETÁ',
            'CRDOBA':'CÓRDOBA',
            'SAN ANDRÉS PROVIDENCIA Y SANTA CATALINA':'SAN ANDRÉS PROV. Y STA. CAT.',
            'BOLVAR':'BOLÍVAR',
            'CHOC':'CHOCÓ'}
df['DEPARTMENT'].replace(dep_mod, inplace=True)

## Análisis exploratorio de datos

### Frecuencia de desviación en costo

In [23]:
# Crear una variable que indique el proyecto tiene desviación en costo
df['COST_DEVIATION'] = df['ADDITIONAL_COST_ORIG_MILLIONS'] > 0

In [24]:
# Tabla de frecuencia de la desviación en costo
tabla_frecuencia(df, 'COST_DEVIATION')

,COST_DEVIATION,TOTAL CONTRATOS,PORCENTAJE
0,False,3535,88.49
1,True,460,11.51


In [25]:
(df['CONTRACT_VALUE_ORIG_MILLIONS']).describe()

count      3995.000000
mean        690.952305
std        5026.682159
min           0.346950
25%          20.499781
50%          66.000000
75%         158.621934
max      173033.340745
Name: CONTRACT_VALUE_ORIG_MILLIONS, dtype: float64

In [26]:
# Crear la variable de tamaño del proyecto
df['CONTRACT_VALUE_RANGE'] = pd.cut(df['CONTRACT_VALUE_ORIG_MILLIONS'], bins=[100,150,300,900,120000], include_lowest=True)

In [27]:
# Frecuencia de desviación por tamaño del proyecto
df.groupby(['CONTRACT_VALUE_RANGE'], observed=False)['COST_DEVIATION'].mean()*100

CONTRACT_VALUE_RANGE
(99.999, 150.0]       5.067064
(150.0, 300.0]       19.740260
(300.0, 900.0]       20.748299
(900.0, 120000.0]    31.157270
Name: COST_DEVIATION, dtype: float64

In [28]:
df.groupby(['REGION'], observed=True)['COST_DEVIATION'].mean().sort_values(ascending=False)

REGION
ORINOQUIA    0.208589
ANDINA       0.172475
CARIBE       0.089552
AMAZONIA     0.089286
PACIFICA     0.067653
OTRA         0.056338
Name: COST_DEVIATION, dtype: float64

In [29]:
df.groupby(['MUNICIPALITY_TYPE'], observed=True)['COST_DEVIATION'].mean()

MUNICIPALITY_TYPE
NACIONAL DESCENTRALIZADO    0.333333
Nacional                    0.000000
No Definido                 0.071429
OTHER                       0.069081
TYPE_1                      0.177419
TYPE_2                      0.150485
TYPE_3                      0.391304
TYPE_4                      0.132530
TYPE_5                      0.231214
TYPE_6                      0.137156
Territorial                 0.000000
Name: COST_DEVIATION, dtype: float64

In [30]:
df.groupby(['PROCESS_TYPE_MOD'], observed=True)['COST_DEVIATION'].mean()

PROCESS_TYPE_MOD
CONTRATACIÓN DIRECTA    0.032026
LICITACIÓN              0.287912
MÍNIMA CUANTÍA          0.103860
OTRO                    0.000000
RÉGIMEN ESPECIAL        0.099668
SELECCIÓN ABREVIADA     0.222042
Name: COST_DEVIATION, dtype: float64

In [31]:
df.columns

Index(['CONTRACT_ID', 'ENTITY_NAME', 'DEPARTMENT', 'MUNICIPALITY_TYPE',
       'PROCESS_TYPE', 'CONTRACT_OBJECT', 'OBJETC_DETAIL',
       'ESTIMATED_COST_ORIG', 'CONTRACT_VALUE_ORIG', 'ADDITIONAL_COST_ORIG',
       'FINAL_COST_ORIG', 'YEAR', 'CONTRACT_DATE', 'START_DATE',
       'ORIGINAL_DEADLINE', 'END_DATE', 'ID_ADJUDICACION', 'URLPROCESO',
       'NIT_ENTIDAD', 'ID_CONTRATISTA', 'ESTIMATED_COST_NORM',
       'CONTRACT_VALUE_NORM', 'ADDITIONAL_COST_NORM', 'FINAL_COST_NORM',
       'ADDITIONAL_TIME', 'FINAL_DEADLINE', 'PROJECT_INTENSITY_NORM',
       'PROJECT_INTENSITY_ORIG', 'AWARD_GROWTH_NORM', 'AWARD_GROWTH_ORIG',
       'COST_DEVIATION_ORIG', 'COST_DEVIATION_NORM', 'TIME_DEVIATION', 'OWNER',
       'REGION', 'GROUP', 'COST_GROUP_QUANTILE',
       'CONTRACT_VALUE_ORIG_MILLIONS', 'ADDITIONAL_COST_ORIG_MILLIONS',
       'FINAL_COST_ORIG_MILLIONS', 'PROCESS_TYPE_MOD', 'COST_DEVIATION',
       'CONTRACT_VALUE_RANGE'],
      dtype='object')

In [32]:
df[df['PROCESS_TYPE'] == 'LICITACIÓN']['OBJETC_DETAIL'].to_clipboard()

In [33]:
df['TIPO_OBRA_CONSTRUCCION'] = df['OBJETC_DETAIL'].str.contains('CONSTRUCCION|PAVIMENTACION|COSNTRUCCION|COSNTRUCCION|CONTRUCCION')
df['TIPO_OBRA_CONSTRUCCION'].sum() 

876

In [34]:
df['TIPO_OBRA_MANTENIMIENTO'] = df['OBJETC_DETAIL'].str.contains('MANTENIMIENTO')
df['TIPO_OBRA_MANTENIMIENTO'].sum() 

1104

In [35]:
df['TIPO_OBRA_REHABILITACION'] = df['OBJETC_DETAIL'].str.contains('REHABILITACION')
df['TIPO_OBRA_REHABILITACION'].sum() 

239

In [36]:
df['TIPO_OBRA_MEJORAMIENTO'] = df['OBJETC_DETAIL'].str.contains('MEJORAMIENTO')
df['TIPO_OBRA_MEJORAMIENTO'].sum() 

1930

In [37]:
df['TOTAL_CLASS_TIPO'] = np.int32(df['TIPO_OBRA_CONSTRUCCION'] + df['TIPO_OBRA_MANTENIMIENTO'] + df['TIPO_OBRA_REHABILITACION'] + df['TIPO_OBRA_MEJORAMIENTO'])

In [38]:
df['TOTAL_CLASS_TIPO'].value_counts()

TOTAL_CLASS_TIPO
1    3442
0     553
Name: count, dtype: int64

In [39]:
df['TIPO_OBRA_OTROS'] = df['TOTAL_CLASS_TIPO'] == 0
df['TIPO_OBRA_OTROS'].sum() 

553

In [40]:
df.columns[pd.Series(df.columns).str.contains('TIPO_OBRA')]

Index(['TIPO_OBRA_CONSTRUCCION', 'TIPO_OBRA_MANTENIMIENTO',
       'TIPO_OBRA_REHABILITACION', 'TIPO_OBRA_MEJORAMIENTO',
       'TIPO_OBRA_OTROS'],
      dtype='object')

In [41]:
cols_tipo_obra = ['TIPO_OBRA_CONSTRUCCION', 'TIPO_OBRA_MANTENIMIENTO',
       'TIPO_OBRA_REHABILITACION', 'TIPO_OBRA_MEJORAMIENTO',
       'TIPO_OBRA_OTROS']
df['TYPE_WORK'] = ''
for i in cols_tipo_obra:
    df.loc[df[i] == True, 'TYPE_WORK'] = i

In [42]:
df.groupby(['TYPE_WORK'])['COST_DEVIATION'].mean()

TYPE_WORK
TIPO_OBRA_CONSTRUCCION      0.182371
TIPO_OBRA_MANTENIMIENTO     0.125554
TIPO_OBRA_MEJORAMIENTO      0.076684
TIPO_OBRA_OTROS             0.146474
TIPO_OBRA_REHABILITACION    0.146893
Name: COST_DEVIATION, dtype: float64

In [43]:
# Gráfica de intervalos
# Asumiendo que df es tu DataFrame y cat_cols son tus variables categóricas de interés
cat_cols = ['CONTRACT_VALUE_RANGE','REGION','MUNICIPALITY_TYPE','PROCESS_TYPE_MOD','TYPE_WORK']
y_var = 'COST_DEVIATION'

resumen = pd.DataFrame()

# Función para calcular la desviación estándar para una variable binaria
def std_binaria(p, n):
    return np.sqrt(p * (1 - p) / n)

# Función para calcular el intervalo de confianza del 95% para una proporción
def confianza_intervalo(p, n):
    z = 1.96  # Valor z para un intervalo de confianza del 95%
    margin = z * std_binaria(p, n)
    return margin

# Inicializar la lista para guardar los resultados de la prueba Chi-cuadrado
chi2_resultados = []

for cat in cat_cols:
    # Agrupar por cat, calcular tamaño y proporción
    temp = (
        df
        .groupby([cat])[y_var]
        .agg(['size','sum','mean'])
        .rename(columns={'size': 'Total Contratos', 'sum':'Total Contratos con Desviación','mean': 'Proporción de Desviación en Costo'})
    )
    
    # Calcular desviación estándar para la variable binaria y los intervalos de confianza
    temp['Desv. Estandar'] = temp.apply(lambda row: std_binaria(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    temp['CI 95% Inferior'] = temp['Proporción de Desviación en Costo'] - temp.apply(lambda row: confianza_intervalo(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    temp['CI 95% Superior'] = temp['Proporción de Desviación en Costo'] + temp.apply(lambda row: confianza_intervalo(row['Proporción de Desviación en Costo'], row['Total Contratos']), axis=1)
    
    # Ajustar valores IC Inferior < 0 e IC Superior > 1
    temp['CI 95% Inferior'] = temp['CI 95% Inferior'].apply(lambda x: np.max([0,x]))
    temp['CI 95% Superior'] = temp['CI 95% Superior'].apply(lambda x: np.min([1,x]))

    
    # Ordenar solo variables categóricas nominales
    if cat in ['REGION','PROCESS_TYPE_MOD','TYPE_WORK']:
       temp =  temp.sort_values(by='Total Contratos', ascending=False)
    
    # Preparar los datos para la prueba Chi-cuadrado
    crosstab = pd.crosstab(df[cat], df[y_var])
    
    # Realizar la prueba Chi-cuadrado y obtener el p-valor
    chi2, p, dof, ex = stats.chi2_contingency(crosstab)
    #p = np.round(p,6)
    chi2_resultados.append((cat, chi2, p))

    
    # Reiniciar el índice para convertir el índice en una columna y facilitar la concatenación
    #temp.reset_index(inplace=True)
    temp['Variable Categorica'] = cat
    #temp['Chi2'] = chi2
    temp['p-value (chi2)'] = p
    
    # Concatenar el DataFrame temporal con el resumen
    resumen = pd.concat([resumen, temp])

# Ajustar el DataFrame resumen
resumen['Categorias'] = resumen.index
resumen = resumen.set_index(['Variable Categorica', 'p-value (chi2)','Categorias'])
resumen

/var/folders/yv/nqf1_stj1lsb93l9v9j6z2h00000gn/T/ipykernel_20991/3575603736.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby([cat])[y_var]


Total Contratos  \
Variable Categorica  p-value (chi2) Categorias                                  
CONTRACT_VALUE_RANGE 1.164537e-26   (99.999, 150.0]                       671   
                                    (150.0, 300.0]                        385   
                                    (300.0, 900.0]                        294   
                                    (900.0, 120000.0]                     337   
REGION               4.175779e-26   ANDINA                               1693   
                                    OTRA                                 1420   
                                    PACIFICA                              473   
                                    ORINOQUIA                             163   
                                    CARIBE                                134   
                                    AMAZONIA                              112   
MUNICIPALITY_TYPE    2.427067e-21   NACIONAL DESCENTRALIZADO                3   
                                    Nacional                               60   
                                    No Definido                            14   
                                    OTHER                                1491   
                                    TYPE_1                                 62   
                                    TYPE_2                                206   
                                    TYPE_3                                 46   
                                    TYPE_4                                 83   
                                    TYPE_5                                173   
                                    TYPE_6                               1779   
                                    Territorial                            78   
PROCESS_TYPE_MOD     1.263691e-64   CONTRATACIÓN DIRECTA                 1530   
                                    MÍNIMA CUANTÍA                       1088   
                                    SELECCIÓN ABREVIADA                   617   
                                    LICITACIÓN                            455   
                                    RÉGIMEN ESPECIAL                      301   
                                    OTRO                                    4   
TYPE_WORK            2.565814e-13   TIPO_OBRA_MEJORAMIENTO               1930   
                                    TIPO_OBRA_MANTENIMIENTO               677   
                                    TIPO_OBRA_CONSTRUCCION                658   
                                    TIPO_OBRA_OTROS                       553   
                                    TIPO_OBRA_REHABILITACION              177   

                                                              Total Contratos con Desviación  \
Variable Categorica  p-value (chi2) Categorias                                                 
CONTRACT_VALUE_RANGE 1.164537e-26   (99.999, 150.0]                                       34   
                                    (150.0, 300.0]                                        76   
                                    (300.0, 900.0]                                        61   
                                    (900.0, 120000.0]                                    105   
REGION               4.175779e-26   ANDINA                                               292   
                                    OTRA                                                  80   
                                    PACIFICA                                              32   
                                    ORINOQUIA                                             34   
                                    CARIBE                                                12   
                                    AMAZONIA                                              10   
MUNICIPALITY_TYPE    2.427067e-21   NACIONAL DESCENTRALIZADO                               1   
                                    Nacional               